## 3 boites en GS :

  - box1 : 376 740 124 585
  - box2 : 736 1099 123 574
  - box3 : 1096 1457 122 565

In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import os
import glob

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Autosaving every 60 seconds


In [2]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/powerspec/powerspec")
import powerspec as pp

In [3]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData
import WavenumberSpectrum as ws


In [4]:
# Some functions that allow us to compute spectra in boxes

##########################################
def get_values_in_box(imin,imax,jmin,jmax,data):
    values = data[:,0,jmin:jmax+1,imin:imax+1]
    values = ma.masked_invalid(values)
    return values


##########################################
def compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,name,WaveSpecResult,param):
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)

##########################################
def compute_spec(data,navlon,navlat):
    days = len(data)
    mth_pspec = []
    for it in np.arange(0,days):
        arr = data[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
        mth_pspec.append(pspec)
    mthly_pspec = np.array(mth_pspec)
    mean_mthly_pspec = mthly_pspec.mean(axis=0)
    return kstep,mean_mthly_pspec

In [5]:
def compute_uspec(case,box):
    
    if box == 'box1':
        imin=376
        imax=740
        jmin=124
        jmax=585
    if box == 'box2':
        imin=736
        imax=1099
        jmin=123
        jmax=574
    if box == 'box3':
        imin=1096
        imax=1457
        jmin=122
        jmax=565

    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/store/colombo/GS36/GS36-'+str(case)+'-S/1d/2012/GS36-'+str(case)+'_y'+str(YrMth[ii])+'d??.1d_gridU.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[0,jmin:jmax+1,imin:imax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[jmin:jmax+1,imin:imax+1]
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [6]:
def compute_vspec(case,box):
    
    if box == 'box1':
        imin=376
        imax=740
        jmin=124
        jmax=585
    if box == 'box2':
        imin=736
        imax=1099
        jmin=123
        jmax=574
    if box == 'box3':
        imin=1096
        imax=1457
        jmin=122
        jmax=565

    param='Vspec'
    varname = 'vomecrty'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']

    for ii in np.arange(12):
        Mth = YrMth[ii]
        if not os.path.exists(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz'):
            print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
            filename='/store/colombo/GS36/GS36-'+str(case)+'-S/1d/2012/GS36-'+str(case)+'_y'+str(YrMth[ii])+'d??.1d_gridV.nc'
            ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
            data = ds[varname]
            if len(ds.nav_lon.dims) == 3:
                navlon=ds.nav_lon[0,jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[0,jmin:jmax+1,imin:imax+1]
            if len(ds.nav_lon.dims) == 2:
                navlon=ds.nav_lon[jmin:jmax+1,imin:imax+1]
                navlat=ds.nav_lat[jmin:jmax+1,imin:imax+1]
            compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,box,WaveSpecResult,param)


In [7]:
def compute_kespec(case,box):
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-Vspec/'
    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-KEspec/'
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        if not os.path.exists(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz'):
            uspec = np.load(u_filenames[i])['pspec']
            vspec = np.load(v_filenames[i])['pspec']
            kspec = np.load(u_filenames[i])['kspec']
            KEspec = 0.5*(uspec + vspec)
            print(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz')
            np.savez(KEspecFolder+'WaveSpec_'+str(box)+'_KEspec_'+YrMth[i]+'.npz',kspec = kspec,KEspec = KEspec)

In [8]:
def check_size(case,box):
    print(case)
    YrMth  = ['2012m01','2012m02','2012m03','2012m04','2012m05','2012m06','2012m07','2012m08','2012m09','2012m10','2012m11','2012m12']
    # Folders containing U and V spectral
    u_database = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-Uspec/'
    v_database = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-Vspec/'
    # Folders to contain computed KE spectral
    KEspecFolder = '/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-'+str(case)+'-KEspec/'
    !mkdir -p $KEspecFolder
    u_filenames = sorted(glob.glob(u_database + 'WaveSpec_'+str(box)+'_Uspec_*.npz'))
    v_filenames = sorted(glob.glob(v_database + 'WaveSpec_'+str(box)+'_Vspec_*.npz'))
    for i in range(len(u_filenames)):
        print(YrMth[i])
        uspec = np.load(u_filenames[i])['pspec']
        print('size uspec = ',uspec.shape)
        vspec = np.load(v_filenames[i])['pspec']
        print('size vspec = ',vspec.shape)


In [9]:
for case in ['MPC001','MPC002','MPC003']:
    compute_uspec(case,'box1')
    compute_vspec(case,'box1')
    compute_kespec(case,'box1')


/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m07.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m08.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m09.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m10.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m11.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-Vspec/WaveSpec_box1_Vspec_2012m12.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-KEspec/WaveSpec_box1_KEspec_2012m01.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-KEspec/WaveSpec_box1_KEspec_2012m02.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-KEspec/WaveSpec_box1_KEspec_2012m03.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC003-KEspec/WaveSpec_box1_KEspec_2012m04.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC

In [10]:
for case in ['MPC001','MPC002','MPC003']:
    check_size(case,'box2')


MPC001
MPC002
MPC003


In [ ]:
for case in ['MPC001','MPC002','MPC003']:
    compute_uspec(case,'box2')
    compute_vspec(case,'box2')
    compute_kespec(case,'box2')


/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m01.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m02.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m03.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m04.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m05.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m06.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m07.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m08.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m09.npz
/scratch/cnt0024/hmg2840/albert7a/GS36-spec/GS36-MPC001-Uspec/WaveSpec_box2_Uspec_2012m10.npz


In [ ]:
for case in ['MPC001','MPC002','MPC003']:
    compute_uspec(case,'box3')
    compute_vspec(case,'box3')
    compute_kespec(case,'box3')
